In [1]:
# Import required libraries
import json
import pandas as pd
import datetime

In [2]:
# Read the patient json file and store it into patients
with open('./dataset/build/patients.json') as f:
    patients = json.load(f)
    
# Read the observation json file and store it into observations
with open('./dataset/build/observations.json') as f1:
    observations = json.load(f1)    
    
# Read the observation json file and store it into organizations
with open('./dataset/build/organizations.json') as f2:
    organizations = json.load(f2)     

In [3]:
# Creating a dataframe to store patient ids and organisation ids
df = pd.DataFrame(columns=['patId', 'orgId'])

In [4]:
# List to create patient ids
patIds = []

# List to create managing Org ids
managingOrgs = []

# Iterate through each entry
for entries in patients['entry']:
    # Store all patient IDs
    patIds.append(entries['resource']['id'])
    
    # Store all org IDs
    managingOrg = entries['resource']['managingOrganization']['reference']
    managingOrgs.append(managingOrg.replace('urn:uuid:',''))

In [5]:
# Display the total number of patient ids and org ids -- Both should be same
print(len(patIds))
print(len(managingOrgs))

100000
100000


In [6]:
# Creating a dataframe to store patient ids and org id
df['patId'] = patIds
df['orgId'] = managingOrgs

# Display dataframe
df

,patId,orgId
0,00058c64-996b-42de-8692-d9db634ef859,065fbe84-3551-4ec3-98a3-0d1198c3cb55
1,000eff0a-e969-4b00-941f-579519186980,065fbe84-3551-4ec3-98a3-0d1198c3cb55
2,00159b3f-3d73-4164-a06c-8213afa3ef08,065fbe84-3551-4ec3-98a3-0d1198c3cb55
3,001ca35e-9353-47bb-945d-e78eca0f2cd5,065fbe84-3551-4ec3-98a3-0d1198c3cb55
4,00277ad6-0534-4d7a-aec8-c3805c9c6750,065fbe84-3551-4ec3-98a3-0d1198c3cb55
...,...,...
99995,ffdc058c-7a8e-4ed0-8255-68f8b16f6e44,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
99996,ffe1dbc1-a5b0-4732-8b7e-6a3336957475,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
99997,ffeb4e99-67de-4e22-9538-340728809706,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
99998,fff4a29e-a0ab-4a76-bc7c-ef2020868900,ff906c99-cc6d-4ca3-ade7-5a45b878c86e


In [7]:
# Creating a dataframe to store patient ids, smoking code and effective datetime from observations
df1 = pd.DataFrame(columns=['patId', 'code', 'effectiveDate'])

In [8]:
# List to create patient ids
patIds = []

# List to create code lists
codeList = []

# List to create dateTime lists
dateTimeList = []

# Iterate through each entry
for entries in observations['entry']:
    # Get the code of each coding value
    smokingStatus = entries['resource']['code']['coding']
    # Iterate through each code
    for item in smokingStatus:
        # Check if it is related to tobacco smoking code
        if(item['code']=='72166-2'):
            # Get the actual smoking status code list
            smokeCode = entries['resource']['valueCodeableConcept']['coding']
            # Get the effective date and time associated with that entry
            dateTimeList.append(entries['resource']['effectiveDateTime'])
            # Get the smoking status code and store in the codeList
            for code in smokeCode:
                codeList.append(code['code'])
            # Get relevant patient id
            patId = entries['resource']['subject']['reference']
            patIds.append(patId.replace('urn:uuid:',''))

In [9]:
# Creating a dataframe to store patient ids and org id
df1['patId'] = patIds
df1['code'] = codeList
df1['effectiveDate'] = dateTimeList

# Display dataframe
df1

,patId,code,effectiveDate
0,000eff0a-e969-4b00-941f-579519186980,266919005,2014-03-02T10:12:53-05:00
1,00159b3f-3d73-4164-a06c-8213afa3ef08,266919005,2017-03-05T10:12:53-05:00
2,001ca35e-9353-47bb-945d-e78eca0f2cd5,266919005,2010-04-11T11:12:53-04:00
3,00277ad6-0534-4d7a-aec8-c3805c9c6750,266919005,2011-02-27T10:12:53-05:00
4,002fdef3-839f-4a0d-b728-f2c15991532f,266919005,2009-04-05T11:12:53-04:00
...,...,...,...
78223,fedfa571-acb9-4873-82f3-5f7f6197d51f,266919005,2010-04-11T07:28:59-04:00
78224,ff31124b-791d-4377-abee-98d01fd88434,266919005,2018-04-28T15:12:25-04:00
78225,ff68b746-ffe1-4087-b7a1-95e9f91c711d,8517006,1975-06-22T02:23:19-04:00
78226,ffae2d79-5761-44f0-ad5a-cfd66ebc64bd,266919005,2009-04-05T07:28:59-04:00


In [10]:
# Split the string to form date and time columns
dateTimeDF = df1["effectiveDate"].str.split("T", n = 1, expand = True)
df1["effectiveDate"] = dateTimeDF[0]
df1["effectiveTime"] = dateTimeDF[1]
timeDF = df1["effectiveTime"].str.split("-", n = 1, expand = True)
df1["effectiveTime"] = timeDF[0]

In [11]:
# Combine the date and time columns to get date time format and drop original date time columns
df1['effectiveDateTime'] = df1['effectiveDate'] + ' ' + df1['effectiveTime']
# Remove two columns names effectiveDate and effectiveTime
df1 = df1.drop(['effectiveDate', 'effectiveTime'], axis = 1) 
df1['effectiveDateTime'] = pd.to_datetime(df1['effectiveDateTime'], format='%Y-%m-%d %H:%M:%S') 

# Display the dataframe
df1

,patId,code,effectiveDateTime
0,000eff0a-e969-4b00-941f-579519186980,266919005,2014-03-02 10:12:53
1,00159b3f-3d73-4164-a06c-8213afa3ef08,266919005,2017-03-05 10:12:53
2,001ca35e-9353-47bb-945d-e78eca0f2cd5,266919005,2010-04-11 11:12:53
3,00277ad6-0534-4d7a-aec8-c3805c9c6750,266919005,2011-02-27 10:12:53
4,002fdef3-839f-4a0d-b728-f2c15991532f,266919005,2009-04-05 11:12:53
...,...,...,...
78223,fedfa571-acb9-4873-82f3-5f7f6197d51f,266919005,2010-04-11 07:28:59
78224,ff31124b-791d-4377-abee-98d01fd88434,266919005,2018-04-28 15:12:25
78225,ff68b746-ffe1-4087-b7a1-95e9f91c711d,8517006,1975-06-22 02:23:19
78226,ffae2d79-5761-44f0-ad5a-cfd66ebc64bd,266919005,2009-04-05 07:28:59


In [12]:
# Sort the dataframe based on ascending order of patient id and descending order of effective datetime and reset the index
df1 = df1.sort_values(['patId', 'effectiveDateTime'], ascending=[True, False])
df1 = df1.reset_index(drop=True)

# Display the dataframe
df1

,patId,code,effectiveDateTime
0,00002ef2-e29a-433b-91cc-4c7516aea2f1,266919005,2016-08-24 08:39:49
1,0001af34-fb61-484f-9382-3d26639f4283,8517006,2017-02-24 07:04:24
2,00050350-bfdc-478c-a039-c2168c47dff4,266919005,1983-01-30 16:55:04
3,0005b8a8-ec04-4db4-bae9-4faad77d4b0a,8517006,2013-01-16 20:06:18
4,0005b8a8-ec04-4db4-bae9-4faad77d4b0a,8517006,2012-12-26 20:06:18
...,...,...,...
78223,fff85ace-18d4-46e7-8060-8e7434b93e1e,266919005,2009-08-16 19:35:39
78224,fffaefd0-5f49-493c-b3f5-d462b09616d1,266919005,2010-03-09 02:54:44
78225,fffbcc52-4069-4e12-9e94-0718a87efce7,266919005,2017-07-01 18:29:30
78226,fffbdcfe-0cd6-487d-b44d-53eb34de9e6a,8517006,2017-06-23 04:06:49


In [13]:
# Group the dataframe based on patient id and filter for the count of codes greater than 2 and reset the index
dfForPatIds = df1.groupby('patId', as_index = False).count()
dfForPatIds = dfForPatIds[dfForPatIds['code']>=2].reset_index(drop=True)

# Get the patient ids with codes>=2 and convert into list
patIdsList = dfForPatIds['patId'].tolist()

# Filter the dataframe based on above patient ids
df1 = df1[df1.patId.isin(patIdsList)].reset_index(drop=True)

In [14]:
# Convert the dataframe entries into dict to analyse further based on the type of codes
patCodeTimeDict = df1.groupby('patId').apply(lambda dfg: dfg.drop('patId', axis=1).to_dict(orient='list')).to_dict()

In [15]:
# Smoking codes list excluding never smoker
smokingCodes1 = ['449868002', '428041000124106', '8517006', '77176002', '266927001', '428071000124103', '428061000124105']

# Smoking codes list excluding unknown if ever smoked
smokingCodes2 = ['449868002', '428041000124106', '8517006', '77176002', '266919005', '428071000124103', '428061000124105']

# List to create a patient ids list which have discrepancy
discrepancyPatIdsList = []   

# Iterate through each key value pair in dict
for key,val in patCodeTimeDict.items():
    # If all codes are never smoker or unknown if ever smoked, then continue
    if all(code == '266919005' for code in val['code']) or all(code == '266927001' for code in val['code']):
        continue
    else:
        # If all the codes are same, continue
        if all(code == val['code'][0] for code in val['code']):
            continue
        else:
            # If latest entry is never smoked and others are either of the smoking codes, add the patient id to discrepancy list
            if val['code'][0] == '266919005' and all(val['code'][i] in smokingCodes1 for i in range(1,len(val['code']))):
                discrepancyPatIdsList.append(key)
            else:
                # If latest entry is unknown if smoked status and others are either of the smoking codes, add the patient id to discrepancy list
                if val['code'][0] == '266927001' and all(val['code'][i] in smokingCodes2 for i in range(1,len(val['code']))):
                    discrepancyPatIdsList.append(key)
                else:
                    continue

In [16]:
# List to create orgIds
orgIdsList = []

# Iterate through each entry
for entries in organizations['entry']:
    # Store all patient IDs
    orgIdsList.append(entries['resource']['id'])

In [17]:
# Creating a dataframe to store organisation ids from organization json file
orgDF = pd.DataFrame(columns=['orgId'])
orgDF['orgId'] = orgIdsList

# Display the dataframe
orgDF

,orgId
0,065fbe84-3551-4ec3-98a3-0d1198c3cb55
1,09f6bac8-3dbb-4848-abc3-d6cb6dab9ff0
2,0a6ca38f-1773-4cc6-a70a-fb89cb113141
3,0d8f1ddb-d146-47c4-8466-ab32d3ae462d
4,10283042-6cc2-43f6-a026-f4547ead69ed
...,...
95,edfbd832-fb71-427d-97c7-e9b100acff44
96,f933e373-a173-425d-b533-f5857d8da94a
97,fb35ab7f-f813-446f-bd8e-642c2c4944fa
98,fdccc30d-94dd-4fcb-b212-d380f2a8decb


In [18]:
# Filter for all the patient ids in discrepancy list and reset index
discrepancyPatDF = df[df['patId'].isin(discrepancyPatIdsList)]
discrepancyPatDF = discrepancyPatDF.reset_index(drop=True)

# Display the dataframe
discrepancyPatDF

,patId,orgId
0,051a86e9-00f1-491e-afbc-87a1a670c880,065fbe84-3551-4ec3-98a3-0d1198c3cb55
1,0935be4f-d61a-49cf-9a51-13208e79370d,065fbe84-3551-4ec3-98a3-0d1198c3cb55
2,11eb4a5b-8780-4286-8d93-f881c2037db5,065fbe84-3551-4ec3-98a3-0d1198c3cb55
3,1985a3a3-8004-4fc7-b70b-a7f8e414d0ee,065fbe84-3551-4ec3-98a3-0d1198c3cb55
4,1d6a3136-8be5-4206-b56f-dc8a19bd4e50,065fbe84-3551-4ec3-98a3-0d1198c3cb55
...,...,...
1850,daa91043-4f27-4309-824b-7e60a4fc5c53,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
1851,dec31bd7-0562-4ece-8962-5704f962a8fc,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
1852,e71b9437-b073-48a3-ba2d-97c2d68341f9,ff906c99-cc6d-4ca3-ade7-5a45b878c86e
1853,ee8cdd87-f68f-47b8-b203-16f257120e4e,ff906c99-cc6d-4ca3-ade7-5a45b878c86e


In [19]:
# Group by the org ID and get the count of patient ids
discrepancyPatDF = discrepancyPatDF.groupby('orgId', as_index = False).count()

# Renaming the columns as orgId and count
discrepancyPatDF.columns = ['orgId', 'count']

# Display the dataframe
discrepancyPatDF

,orgId,count
0,065fbe84-3551-4ec3-98a3-0d1198c3cb55,37
1,09f6bac8-3dbb-4848-abc3-d6cb6dab9ff0,20
2,1152411e-c5a1-4753-bc27-eac39beaefc1,24
3,19e5d812-81d4-4313-a3f2-4c2804a7d787,5
4,1a6bc667-8709-4a57-af38-a48c0d8b698d,6
...,...,...
70,edc80912-a016-4093-943d-3a452512934e,1
71,edfbd832-fb71-427d-97c7-e9b100acff44,3
72,f933e373-a173-425d-b533-f5857d8da94a,1
73,fdccc30d-94dd-4fcb-b212-d380f2a8decb,141


In [20]:
# Join the two dataframes above with common column orgId
orgCountDF = orgDF.join(discrepancyPatDF.set_index('orgId'), on='orgId')

# Display the dataframe
orgCountDF

,orgId,count
0,065fbe84-3551-4ec3-98a3-0d1198c3cb55,37.0
1,09f6bac8-3dbb-4848-abc3-d6cb6dab9ff0,20.0
2,0a6ca38f-1773-4cc6-a70a-fb89cb113141,NaN
3,0d8f1ddb-d146-47c4-8466-ab32d3ae462d,NaN
4,10283042-6cc2-43f6-a026-f4547ead69ed,NaN
...,...,...
95,edfbd832-fb71-427d-97c7-e9b100acff44,3.0
96,f933e373-a173-425d-b533-f5857d8da94a,1.0
97,fb35ab7f-f813-446f-bd8e-642c2c4944fa,NaN
98,fdccc30d-94dd-4fcb-b212-d380f2a8decb,141.0


In [21]:
# Replace the NaN values with 0
orgCountDF['count'].fillna(0, inplace=True)

# Convert the count column into integer
orgCountDF['count'] = orgCountDF['count'].astype(int)

# Display the final dataframe
orgCountDF

,orgId,count
0,065fbe84-3551-4ec3-98a3-0d1198c3cb55,37
1,09f6bac8-3dbb-4848-abc3-d6cb6dab9ff0,20
2,0a6ca38f-1773-4cc6-a70a-fb89cb113141,0
3,0d8f1ddb-d146-47c4-8466-ab32d3ae462d,0
4,10283042-6cc2-43f6-a026-f4547ead69ed,0
...,...,...
95,edfbd832-fb71-427d-97c7-e9b100acff44,3
96,f933e373-a173-425d-b533-f5857d8da94a,1
97,fb35ab7f-f813-446f-bd8e-642c2c4944fa,0
98,fdccc30d-94dd-4fcb-b212-d380f2a8decb,141


In [22]:
# Reading the input file to store N and test cases
with open('./input_files/Q2/input.txt', 'r') as inputFile:
    inputText = inputFile.readlines()
    for i in range(0,len(inputText)):
        inputText[i] = inputText[i].replace('\n', '')

In [23]:
# Take input from the user for the number of entries
N = int(inputText[0])

# Create an empty list to store user entered org ids
countList = []

# Take patient ids from the user for the number of times the user entered above
for i in range(1,len(inputText)):
    countList.append(orgCountDF[orgCountDF['orgId']==inputText[i]]['count'].iat[0])

In [24]:
# Store the resultant output in a txt file
with open('./problemBOutput.txt', 'w') as f:
    for i in range(0,len(countList)):
        text = "Test " + str(i+1) + ": " + str(countList[i]) + "\n"
        f.write(text)